In [1]:
import os
import sys
from tqdm import tqdm
sys.path.insert(0, '../')

from colbert.infra import Run, RunConfig, ColBERTConfig
from colbert.data import Queries, Collection
from colbert import Indexer, Searcher

nbits = 2   # encode each dimension with 2 bits
doc_maxlen = 300   # truncate passages at 300 tokens

checkpoint = 'downloads/colbertv2.0'
index_name = f'lifestyle.dev.2bits'
dataroot = 'downloads/lotte'
dataset = 'lifestyle'
datasplit = 'dev'

queries = os.path.join(dataroot, dataset, datasplit, 'questions.search.tsv')
collection = os.path.join(dataroot, dataset, datasplit, 'collection.tsv')

queries = Queries(path=queries)
collection = Collection(path=collection)

# f'Loaded {len(queries)} queries and {len(collection):,} passages'
# To create the searcher using its relative name (i.e., not a full path), set
# experiment=value_used_for_indexing in the RunConfig.
with Run().context(RunConfig(experiment='notebook')):
    searcher = Searcher(index=index_name)


# If you want to customize the search latency--quality tradeoff, you can also supply a
# config=ColBERTConfig(ncells=.., centroid_score_threshold=.., ndocs=..) argument.
# The default settings with k <= 10 (1, 0.5, 256) gives the fastest search,
# but you can gain more extensive search by setting larger values of k or
# manually specifying more conservative ColBERTConfig settings (e.g. (4, 0.4, 4096)).

file_path = '/home/adqaicp/CS-M1/TTDS/Coursework1/id_pid.tsv'
with open(file_path, "r", encoding="utf-8") as f:
    lines = f.readlines()
# use dict to store id and pid
id_pid = {}
for line in lines:
    line = line.strip()
    line = line.split('\t')
    id_pid[line[0]] = line[1]

[Feb 24, 22:09:27] #> Loading the queries from downloads/lotte/lifestyle/dev/questions.search.tsv ...
[Feb 24, 22:09:27] #> Got 417 queries. All QIDs are unique.

[Feb 24, 22:09:27] #> Loading collection...
0M 1M 2M 
[Feb 24, 22:09:32] #> Loading collection...
0M 1M 2M 
[Feb 24, 22:09:42] #> Loading codec...
[Feb 24, 22:09:42] Loading decompress_residuals_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 24, 22:09:43] Loading packbits_cpp extension (set COLBERT_LOAD_TORCH_EXTENSION_VERBOSE=True for more info)...
[Feb 24, 22:09:44] #> Loading IVF...
[Feb 24, 22:09:45] #> Loading doclens...


100%|██████████| 95/95 [00:00<00:00, 1030.71it/s]


[Feb 24, 22:09:45] #> Loading codes and residuals...


100%|██████████| 95/95 [00:05<00:00, 16.94it/s]


In [2]:
from http.server import HTTPServer, BaseHTTPRequestHandler
import json

class ColBERTTest(BaseHTTPRequestHandler):

    def do_POST(self):
        self.send_response(201)
        self.send_header('Content-type', 'application/json; charset=utf-8')
        self.end_headers()
        content_length = int(self.headers['Content-Length'])

        body = self.rfile.read(content_length).decode('utf8')
        data = json.loads(body)
        # print(data)

        # >>> your codes goes here, replace the random numbers with search results >>>
        query = data['query']
        k = int(data['k'])
        results = searcher.search(query, k=k)
        # query = "Chinese cooking recepit"
        # k = 3
        # print(f"#> {query}")
        rank_info_books = {}
        for passage_id, passage_rank, passage_score in zip(*results):
            book_id = id_pid[str(passage_id)]
            rank_info_books[book_id] = "{:.1f}".format(passage_score)
        response = {'rank_info_books': rank_info_books}
        # <<< @Xiaochen Zhang <<<

        jstring = json.dumps(response, ensure_ascii=False).encode('utf-8')
        self.wfile.write(jstring)


if __name__ == "__main__":
    httpd = HTTPServer(('localhost', 30003), ColBERTTest)
    httpd.serve_forever()

127.0.0.1 - - [24/Feb/2023 22:10:19] "POST /colbert HTTP/1.1" 201 -


{'query': 'learn SQL', 'k': 5}

#> QueryTokenizer.tensorize(batch_text[0], batch_background[0], bsize) ==
#> Input: . learn SQL, 		 True, 		 None
#> Output IDs: torch.Size([32]), tensor([  101,     1,  4553, 29296,   102,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103,   103,   103,   103,   103,   103,   103,   103,   103,
          103,   103])
#> Output Mask: torch.Size([32]), tensor([1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])



127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -


{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}


127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:20] "POST /colbert HTTP/1.1" 201 -


...

127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -


{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}


127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -
127.0.0.1 - - [24/Feb/2023 22:10:32] "POST /colbert HTTP/1.1" 201 -


{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}
{'query': 'learn SQL', 'k': 5}


KeyboardInterrupt: 